1) Faça um programa que transforme a sequencia no reverso complementar. Ou seja, seria o equivalente a ver como seria a fita de DNA complementar a esta sequencia. OBS: O programa tem que deixar o usuário submeter um arquivo fasta após o programa ser iniciado! (Em perl seria o ARGV). A saída é um arquivo fasta contendo o reverso complementar e no header tem que estar o nome original e no final "-complement_reverse".

In [23]:
DNA_TO_COMPLEMENTAR = {"A" : "T", "T" : "A", "C" : "G", "G" : "C"}

# FILE_INPUT = str(input("Path do arquivo a ser lido"))
FILE_INPUT = "../Files/sequence.fasta"
FILE_OUTPUT = "../Answers/Ex4_1.fasta"

def complementary_fasta(line, mapping):
    new_line = ""
    print(line)
    for char in line:
        if char != "\n":
            new_line += mapping[char]
    return new_line + "\n"
        
with open(FILE_INPUT, "r") as f:
    sense_seq = f.readlines()

with open(FILE_OUTPUT, "w") as j:
    j.write(str(sense_seq[0]).rstrip("\n") + "-complement_reverse" + "\n")
    for line in sense_seq[1:]:
        j.write(complementary_fasta(line,DNA_TO_COMPLEMENTAR))  

with open(FILE_OUTPUT, "r") as k: 
    antisense_seq = k.readlines()    

...

TGCC





2) ORFinder. Faça um programa que busque por todas as ORFs na sequência original no frame + 1 (comece na primeira base e vá andando de 3 em 3, para cada possível codon). O arquivo de saída são 2 arquivos multi-fastas:
2.1) ORF_dna: contém as sequencias de nucleotídeos de todas as ORFs onde no header estará o nome original + início da ORF + o fim da ORF + tamanho em nt da ORF (separados por "-").
2.2) ORF_aa: contém as sequencias de aminoácidos de todas as ORFs onde no header estará o nome original + tamanho da ORF em aa

3) Repita o programa anterior mas procure por ORFs nos frames +1, +2 e +3. Informe no header em que frame a ORF estava.

4) Repita o programa anterior mas agora inclua os frames -1, -2 e -3 (ou seja, buscando na sequencia complementar reverso)


In [84]:
import textwrap

DNA_TO_RNA = {"A" : "U", "T" : "A", "C" : "G", "G" : "C"}
RNA_TO_PROTEIN = {"UUU":"F", "UUC":"F", "UUA":"L", "UUG":"L",
    "UCU":"S", "UCC":"S", "UCA":"S", "UCG":"S",
    "UAU":"Y", "UAC":"Y", "UAA":"*", "UAG":"*",
    "UGU":"C", "UGC":"C", "UGA":"*", "UGG":"W",
    "CUU":"L", "CUC":"L", "CUA":"L", "CUG":"L",
    "CCU":"P", "CCC":"P", "CCA":"P", "CCG":"P",
    "CAU":"H", "CAC":"H", "CAA":"Q", "CAG":"Q",
    "CGU":"R", "CGC":"R", "CGA":"R", "CGG":"R",
    "AUU":"I", "AUC":"I", "AUA":"I", "AUG":"M",
    "ACU":"T", "ACC":"T", "ACA":"T", "ACG":"T",
    "AAU":"N", "AAC":"N", "AAA":"K", "AAG":"K",
    "AGU":"S", "AGC":"S", "AGA":"R", "AGG":"R",
    "GUU":"V", "GUC":"V", "GUA":"V", "GUG":"V",
    "GCU":"A", "GCC":"A", "GCA":"A", "GCG":"A",
    "GAU":"D", "GAC":"D", "GAA":"E", "GAG":"E",
    "GGU":"G", "GGC":"G", "GGA":"G", "GGG":"G",}

def ORF_dna_aa(sequence, frame, rna, aa):
    formated_sequence="".join(sequence).replace("\n","")
    sequence_rna =""
    ORFs_found = []
    for i in formated_sequence:
        sequence_rna += rna[i]
    SEQUENCE_LEN = len(formated_sequence)
    START_SEARCH = abs(frame) - 1
    LAST_SENSE_CODON = (SEQUENCE_LEN - START_SEARCH) % 3
    START_CODON = "AUG"
    STOP_CODONS =["UAG", "UAA", "UGA"]
    New_ORF = None
    for j in range(START_SEARCH, (SEQUENCE_LEN - LAST_SENSE_CODON) , 3):
        if (New_ORF):
            if New_ORF["started"] == "yes":
                New_ORF["sequence_dna"] += f"{str(formated_sequence[j:j+3])} "
                New_ORF["sequence_aa"] += str(aa.get(sequence_rna[j:j+3]))
            if (sequence_rna[j:j+3] in STOP_CODONS):
                New_ORF["end"] = j+3
                New_ORF["len_nt"] = len(New_ORF["sequence_dna"].replace(" ", ""))
                New_ORF["len_aa"] = len(New_ORF["sequence_aa"])
                ORFs_found.append(New_ORF)
                New_ORF = None
        if (sequence_rna[j:j+3] == START_CODON) & (not New_ORF):
            New_ORF = {"init" : j+1,
                       "sequence_dna" : f"{str(formated_sequence[j:j+3])} ",
                       "sequence_aa" : f"{str(aa.get(sequence_rna[j:j+3]))}",
                       "frame" : frame,
                       "started": "yes" }

            
    return ORFs_found

def create_fasta (filepath, sequence, ORFs, mode):
    with open(filepath+"ORF_dna"+".fasta", mode) as f:
        for orf_1 in ORFs:
            for orf in orf_1:
                header = str(sequence[0]).replace("\n","")
                f.write(textwrap.fill(f"{header} - Start:{orf['init']} - End:{orf['end']} - np:{orf['len_nt']} - Frame:{orf['frame']}", width=70) + "\n")
                f.write(textwrap.fill(str(orf['sequence_dna']), width=70) + "\n")
    with open(filepath+"ORF_aa"+".fasta", mode) as l:
        for orf_1 in ORFs:
            for orf in orf_1:
                header = str(sequence[0]).replace("\n","")
                l.write(textwrap.fill(f"{header}- Lenght:{orf['len_aa']} - Frame:{orf['frame']}",width=66) +"\n")
                l.write(textwrap.fill(f"{orf['sequence_aa']}",width=66)+"\n")



Creating files for frame -3,-2,-1,1,2,3

In [85]:
ORFS_sense = [ORF_dna_aa(sense_seq[1:],frame,DNA_TO_RNA,RNA_TO_PROTEIN) for frame in [1,2,3]]
ORFS_antisense = [ORF_dna_aa(antisense_seq[1:],frame,DNA_TO_RNA,RNA_TO_PROTEIN) for frame in [-1,-2,-3]]

create_fasta("../Answers/Ex4_",sense_seq,ORFS_sense, "w")
create_fasta("../Answers/Ex4_",antisense_seq,ORFS_antisense,"a")
